In [3]:
import os
import cv2
import numpy as np
from typing import Dict, Tuple
from skimage import io 

In [13]:
def generateImageSetDict(imageSetNum: str, NIR: bool):
    # Initialize outputs
    imgMask = {}
    imgArray = {}
    
    # Initialize Data Path
    imageType = 'NIR' if NIR else 'RED'
    sampleDir = '/home/mark/DataBank/probav_data/train/'
    imageDir = os.path.join(sampleDir, imageType, imageSetNum)
    if not os.path.exists(imageDir):
        print("Path does not exist")
        return ({}, {})
    
    # Iterate through all items and populate the Dicts
    imageList = sorted(os.listdir(imageDir))
    imgArray['HR'] = io.imread(os.path.join(imageDir, 'HR.png'))
    imgMask['HR'] = io.imread(os.path.join(imageDir, 'SM.png'))
    for img in imageList:
        if img == 'HR.png':
            continue
        elif img == 'SM.png':
            continue
        elif img[0:2] == 'LR':
            imgArray[img.split('.')[0]] = io.imread(os.path.join(imageDir, img))
        else:
            imgMask[img.split('.')[0]] = io.imread(os.path.join(imageDir, img))
    
    return (imgArray, imgMask)        

In [10]:
qMapImgDir = os.path.join(sampleDir, imageList[22])
qMap = cv2.imread(qMapImgDir, 0)
qMap.shape

NameError: name 'sampleDir' is not defined

In [27]:
qMapImgDir

'C:\\Users\\Mark\\Documents\\Projects\\databank\\train\\NIR\\imgset0594\\QM000.png'

In [15]:
x = generateImageSetDict('imgset0888', True)

In [20]:
(x[1]['HR'].astype(np.bool) == False).any()

True

In [21]:
np.unique(x[1]['HR'])

array([  0, 255], dtype=uint8)